In [2]:
# pip install vaex

  Using cached Shapely-1.7.1-cp38-cp38-win_amd64.whl (1.0 MB)
Note: you may need to restart the kernel to use updated packages.Building wheels for collected packages: aplus



  Created wheel for aplus: filename=aplus-0.11.0-py3-none-any.whl size=4415 sha256=3f4659dcaf236c376480378987b7bb537f0da9c9f7a7e46e9d444677eeac5920
  Stored in directory: c:\users\adriano\appdata\local\pip\cache\wheels\de\93\23\3db69e1003030a764c9827dc02137119ec5e6e439afd64eebb
Successfully built aplus
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.5.1
    Uninstalling ipywidgets-7.5.1:
      Successfully uninstalled ipywidgets-7.5.1


In [3]:
import pandas as pd
import re
import vaex
import numpy as np

# Pre-processing 1 

In [4]:
df = vaex.from_csv('original_data.csv',convert=True, chunk_size=500000)
flag = 1

for chunk in df:
# Renaming column
    chunk.rename(columns={'Point-of-Sale_ID': 'POS_ID'}, inplace=True)

# Removing text from strings and keeping just numbers, downcasting to save space
    chunk.ProductCategory_ID = pd.to_numeric(chunk.ProductCategory_ID.str.extract('(\d+)', expand=False), downcast='integer')
    chunk.ProductFamily_ID = pd.to_numeric(chunk.ProductFamily_ID.str.extract('(\d+)', expand=False), downcast='integer')
    chunk.ProductBrand_ID = pd.to_numeric(chunk.ProductBrand_ID.str.extract('(\d+)', expand=False), downcast='integer')
    chunk.ProductName_ID = pd.to_numeric(chunk.ProductName_ID.str.extract('(\d+)', expand=False), downcast='integer')
    chunk.ProductPackSKU_ID = pd.to_numeric(chunk.ProductPackSKU_ID.str.extract('(\d+)', expand=False), downcast='integer')
    chunk.POS_ID = pd.to_numeric(chunk.POS_ID.str.extract('(\d+)', expand=False), downcast='integer')

# Downcasting to save space
    chunk.Value = pd.to_numeric(chunk.Value, downcast='float')

# Convert to datetime
    chunk.Date = pd.to_datetime(chunk.Date, infer_datetime_format=True)  

    # Transform text in binary
    chunk.Measures.replace({'Sell-out units': 1, 'Sell-out values': 0}, inplace=True)
    # Reduce its size
    chunk.Measures = pd.to_numeric(chunk.Measures, downcast='integer')

    # Rename it
    chunk.rename(columns={'Measures': 'isUnit'}, inplace=True)
    
## NEW CODE

# Convert to datetime
#    chunk['Date'] = pd.to_datetime(chunk.Date, infer_datetime_format=True) 
    
# Create Quarter column
    chunk['Quarter'] = chunk.Date.dt.to_period("Q")    

# Create UniqueID column
    chunk['UniqueID'] = pd.Series(chunk['ProductPackSKU_ID'].astype(str)+'_'+
          chunk['Quarter'].astype(str)+'_'+
          chunk['POS_ID'].astype(str))

# Create Unit and NotUnit dataframes and merge them
    Unit = chunk[chunk.isUnit == 1]
    NotUnit = chunk[chunk.isUnit != 1]
    chunk =  pd.merge(Unit, NotUnit[['isUnit','Value','UniqueID']], on="UniqueID")
    chunk.rename({'Value_x': 'Units',
             'Value_y': 'Value'}, axis=1, inplace=True)

    chunk['UnitPrice']=chunk['Value']/chunk['Units']
    chunk.drop(['isUnit_y', 'isUnit_x'], axis=1, inplace=True)

    
# The pipeline above does all the transformations for the whole dataset and saves it to a .csv called "data_new".

    if flag == 1:
            chunk.to_csv('data_new3.csv', mode='a', index=None, header=True)
            flag = 0
    else:
            chunk.to_csv('data_new3.csv', mode='a', index=None, header=None)

RuntimeError: Can't decrement id ref count (file write failed: time = Mon May 17 23:12:20 2021
, filename = 'original_data.csv.hdf5', file descriptor = 4, errno = 28, error message = 'No space left on device', buf = 0000020979C99028, total write size = 65536, bytes this sub-write = 65536, bytes actually written = 18446744073709551615, offset = 4423223796)

# Preprocessing 2 

In [147]:
chunk.groupby(by=['POS_ID','Quarter','ProductPackSKU_ID']).sum()

ProductFamily_ID  ProductCategory_ID  \
POS_ID Quarter ProductPackSKU_ID                                         
1      2016Q1  40                              525                1050   
               42                              336                 672   
               57                            83232              809200   
               61                              288                2800   
               66                            15138              147175   
...                                            ...                 ...   
2      2019Q4  8626                             48                 712   
               8627                            192                2848   
               8629                             12                 178   
               8630                            768               11392   
               8660                              1                 178   

                                  ProductBrand_ID  ProductName_ID    Units  \
POS_ID Quarter ProductPackSKU_ID                                             
1      2016Q1  40                             100             425     50.0   
               42                              64             272     32.0   
               57                           23120           97104  32572.0   
               61                              80             384     40.0   
               66                            4205           22707   5191.0   
...                                           ...             ...      ...   
2      2019Q4  8626                          6108           11364      4.0   
               8627                         24432           45456     16.0   
               8629                          1527            2841      1.0   
               8630                         97728          181824     64.0   
               8660                          1535            2849      3.0   

                                       Value     UnitPrice  
POS_ID Quarter ProductPackSKU_ID                            
1      2016Q1  40                    78700.0  3.935000e+04  
               42                    49600.0  2.480000e+04  
               57                 59574188.0  1.215660e+07  
               61                    73160.0  3.200750e+04  
               66                  9494339.0  3.163786e+06  
...                                      ...           ...  
2      2019Q4  8626                   7272.0  7.272000e+03  
               8627                  40304.0  4.030400e+04  
               8629                    482.0  4.820000e+02  
               8630                  57600.0  5.760000e+04  
               8660                   4950.0  1.650000e+03  

[22146 rows x 7 columns]